In [66]:
import pandas as pd
import numpy as np
import random as rd 
import pickle as pk

from Judge.Score import *   # 评价指标

In [67]:
df_ft = pd.read_csv('./Datasets/train_feature.csv')
df_lb = pd.read_csv('./Datasets/train_label.csv') 

# df_origin = pd.concat([df_ft_origin, df_lb_origin],axis=1)


## 数据预处理
首先进行数据的预处理。

In [68]:
# # print(df_ft_origin.head())
# print(df_ft_origin['feature_0'])
# print(type(df_ft_origin['feature_0']))
# print(df_ft_origin.mean())
# print(df_ft_origin.median())
# print(df_ft_origin.var())



In [69]:
from Preprocessing import Tools as M_TOOL
from Preprocessing import PCA as M_PCA

from itertools import chain as ch


df = pd.concat([df_ft, df_lb], axis=1)
df.fillna(df.median(), inplace=True)

# 删除噪声
df = M_TOOL.Drop_noise(df, if_debug=False)

# 数据归一化
df = M_TOOL.Normalization(df, if_debug=False)


# PCA_model = M_PCA.PCA(df.values)
# df = pd.DataFrame(PCA_model.reduce_dimension())
# print(df.head())
# corr = np.array(df.corr())
# print(df.corr())


# def flat(nest,cond_func=lambda r:type(r)==ast.Num,get_func=lambda r:r.n):
#     cd = json.dumps(nest)
#     t = ast.parse(cd)
#     g = ast.walk(t)
#     arr = list(g)
#     arr = list(filter(cond_func,arr))
#     arr = list(map(get_func,arr))
#     return(arr)

# 作者：navegador
# 链接：https://www.zhihu.com/question/356442472/answer/1585424932
# 来源：知乎
# 著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。



# corr_list = list(ch.from_iterable(corr))

# corr_list = list(set(corr_list).difference(set([corr_list[i] for i in range(len(corr_list)-1,-1,-1) if corr_list[i] == 1.0])))

# corr_list.sort()
# # print(corr_list)

# max_list = corr_list[-5:]
# print(max_list)


    
# sub_df_1 = df[df['label'] == 0]






# df.drop_duplicates(inplace=True)
# print(df.shape)
# print(sub_df_1.head())
# print(sub_df_1.corr())

print(df.shape)
# M_TOOL.Find_useless_feature(df)
df = M_TOOL.Delete_feature(df)
print(df.shape)
print(df.head())
X_train, y_train, X_test, y_test = M_TOOL.random_Split_Data_Label(df, [0, 1])


(5195, 121)
(5195, 13)
   feature_9  feature_35  feature_43  feature_48  feature_67  feature_92  \
0   0.040544    0.003153    0.027659    0.058805    0.014114    0.838889   
1   0.049125    0.010094    0.127033    0.070136    0.002913    0.418519   
2   0.015803    0.144561    0.009585    0.033477    0.021656    0.651852   
3   0.044818    0.074384    0.032242    0.074702    0.035810    0.559259   
4   0.047807    0.020132    0.022263    0.026900    0.012407    0.768519   

   feature_98  feature_99  feature_100  feature_101  feature_106  feature_114  \
0    0.105263    0.036992     0.134971     0.038183        0.125     0.008813   
1    0.105263    0.227899     0.008406     0.059721        0.000     0.038113   
2    0.000000    0.008371     0.022958     0.061517        0.000     0.007205   
3    0.105263    0.017050     0.066100     0.019795        0.125     0.057947   
4    0.000000    0.016713     0.012045     0.081389        0.000     0.040607   

   label  
0      0  
1      0  


In [70]:
# from sklearn.manifold import *

# m = LocallyLinearEmbedding(n_components=80)
# m.fit(X_train, y_train)
# X_train = m.transform(X_train)
# X_test = m.transform(X_test)



## 模型训练与预测

接下来，我们将分别使用线性回归模型、决策树模型、神经网络模型、支持向量机以及 XGBoost 等分类模型对数据集进行训练，并在验证集上进行预测。

In [71]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression, LinearRegression

#递归特征消除法，返回特征选择后的数据
#参数estimator为基模型
#参数n_features_to_select为选择的特征个数
# print(X_train.shape)
# X_new = RFE(estimator=LinearRegression(), n_features_to_select=50).fit_transform(X_train, y_train)
# print(X_new.shape)
# drop_index = M_TOOL.Find_drop_feature(X_train, X_new)
# # print(drop_index)

# X_train = np.delete(X_train, drop_index, axis=1)
# X_test = np.delete(X_test, drop_index, axis=1)





In [72]:
from Methods import LinearReg as M_LR
# 线性回归模型
X_train, y_train, X_test, y_test = M_TOOL.random_Split_Data_Label(df)

model_liner = M_LR.LinearRegression(X_train, y_train)
model_liner.fit()
# model_liner.save()
# model_liner = pk.load(open("./model/Model_LinearReg_2022_12_24_21_07_58.dat", "rb"))    # 使用已保存的模型

pre = model_liner.predict(X_test)



acc = Accuracy(pre, y_test)
print("线性回归模型分类准确度为 {}".format(acc))


# 与 Sklearn 比较
from sklearn.linear_model import LinearRegression as LR  
model_liner_skl = LR().fit(X_train, y_train)
pre_skl = model_liner_skl.predict(X_test)

acc = Accuracy(pre_skl, y_test)
print("SKLearn 分类准确度为 {}".format(acc))


线性回归模型分类准确度为 0.2375096079938509
SKLearn 分类准确度为 0.23981552651806304


In [73]:
# 决策树模型

from Methods import DecTree as M_DTC
X_train, y_train, X_test, y_test = M_TOOL.random_Split_Data_Label(df)

parameters = {
    "criterion": "gini",       # 选择特征的标准，分为 "gini" 和 "entropy"
    "splitter": "best",        # 特征划分标准，分为 "best" 和 "random"
    "max_depth": 11,
    "min_samples_split": 2,   
    "min_samples_leaf": 4,    
    "max_leaf_nodes": 5000,   
    "min_impurity_decrease": 0.0, 
    "if_silent": 1
}

# parameters = {
#     "criterion": "gini",    # 选择特征的标准，分为 "gini" 和 "entropy"
#     "splitter": "best",        # 特征划分标准，分为 "best" 和 "random"
#     "max_depth": 14,
#     "min_samples_split": 2,    # 叶子最小样本数
#     "min_samples_leaf": 4,     # 划分最小样本数
#     "max_leaf_nodes": 10000,    # 最大叶节点个数
#     "min_impurity_decrease": 0.0, # 最小划分不纯度减少量
#     "if_silent": 1
# }

model_dtc = M_DTC.DecisionTree(X_train, y_train)
model_dtc.fit(parameters)

pre = model_dtc.predict(X_test)
pre = np.array(pre).reshape(-1, 1)


acc_dtc = Accuracy(pre, y_test)
print("决策树分类准确度为 {}".format(acc_dtc))


决策树分类准确度为 0.25211375864719443


In [74]:
# parameters = {
#     "criterion": "gini",    # 选择特征的标准，分为 "gini" 和 "entropy"
#     "splitter": "best",        # 特征划分标准，分为 "best" 和 "random"
#     "max_depth": 14,
#     "min_samples_split": 2,    # 叶子最小样本数
#     "min_samples_leaf": 4,     # 划分最小样本数
#     "max_leaf_nodes": 10000,    # 最大叶节点个数
#     "min_impurity_decrease": 0.0, # 最小划分不纯度减少量
#     "if_silent": 1
# }

In [75]:
# from Methods import RegTree as M_RT
# # 回归树模型

# m, n = y_train.shape 
# y_t = np.zeros((m, n))
# model_regtree = M_RT.RegressionTree(X_train, y_train, y_t)
# default_parameters = {
#     "lamda": 2,             # Hyperparameters
#     "gamma": 1e-6,          # Hyperparameters
#     "gain_delta": 0,        # The minimum gain
#     "max_depth": 3,
#     "max_leaves": 100,
#     "max_nodes": 1000,
#     "min_samples": 10,      # Minimum number of samples on a leaf
#     "min_feature_dif": 5,   # The minimum number of different value for current feature
#     "if_silent": 0
# }
# model_regtree.fit(default_parameters)
# pre = model_regtree.predict(X_test)

# acc = Accuracy(pre, Y_test)
# print("回归树模型分类准确度为 {}".format(acc))


In [76]:
# from Methods import SVM as M_SVM
# # 支持向量机模型

# X_train, y_train, X_test, y_test = M_TOOL.random_Split_Data_Label(df, [0, 1])
# model_SVM_01 = M_SVM.SupportVectorMachine(X_train, y_train)
# model_SVM_01.fit(max_times=200, ifsilent=True)
# pre_01 = model_SVM_01.predict(X_test)
# acc_01 = Accuracy(pre_01, y_test)
# print("SVM (01) 模型分类准确度为 {}".format(acc_01))

# X_train, y_train, X_test, y_test = M_TOOL.random_Split_Data_Label(df, [0, 2])
# model_SVM_02 = M_SVM.SupportVectorMachine(X_train, y_train)
# model_SVM_02.fit(max_times=200, ifsilent=True)
# pre_02 = model_SVM_02.predict(X_test)
# acc_02 = Accuracy(pre_02, y_test)
# print("SVM (02) 模型分类准确度为 {}".format(acc_02))

# X_train, y_train, X_test, y_test = M_TOOL.random_Split_Data_Label(df, [0, 3])
# model_SVM_03 = M_SVM.SupportVectorMachine(X_train, y_train)
# model_SVM_03.fit(max_times=200, ifsilent=True)
# pre_03 = model_SVM_03.predict(X_test)
# acc_03 = Accuracy(pre_03, y_test)
# print("SVM (03) 模型分类准确度为 {}".format(acc_03))

# X_train, y_train, X_test, y_test = M_TOOL.random_Split_Data_Label(df, [1, 2])
# model_SVM_12 = M_SVM.SupportVectorMachine(X_train, y_train)
# model_SVM_12.fit(max_times=200, ifsilent=True)
# pre_12 = model_SVM_12.predict(X_test)
# acc_12 = Accuracy(pre_12, y_test)
# print("SVM (12) 模型分类准确度为 {}".format(acc_12))

# X_train, y_train, X_test, y_test = M_TOOL.random_Split_Data_Label(df, [1, 3])
# model_SVM_13 = M_SVM.SupportVectorMachine(X_train, y_train)
# model_SVM_13.fit(max_times=200, ifsilent=True)
# pre_13 = model_SVM_13.predict(X_test)
# acc_13 = Accuracy(pre_13, y_test)
# print("SVM (13) 模型分类准确度为 {}".format(acc_13))

# X_train, y_train, X_test, y_test = M_TOOL.random_Split_Data_Label(df, [2, 3])
# model_SVM_23 = M_SVM.SupportVectorMachine(X_train, y_train)
# model_SVM_23.fit(max_times=200, ifsilent=True)
# pre_23 = model_SVM_23.predict(X_test)
# acc_23 = Accuracy(pre_23, y_test)
# print("SVM (23) 模型分类准确度为 {}".format(acc_23))


# X_train, y_train, X_test, y_test = M_TOOL.random_Split_Data_Label(df)
# pre_01 = model_SVM_01.predict(X_test)
# pre_02 = model_SVM_02.predict(X_test)
# pre_03 = model_SVM_03.predict(X_test)
# pre_12 = model_SVM_12.predict(X_test)
# pre_13 = model_SVM_13.predict(X_test)
# pre_23 = model_SVM_23.predict(X_test)
# print(pre_01)
# print(pre_02)
# print(pre_03)
# print(pre_12)
# print(pre_13)
# print(pre_23)

In [77]:
# # 与 SKLearn 比较
# from sklearn import svm 
# model = svm.SVC(kernel='linear')
# model.fit(X_train, y_train.flatten())
# pre_skl = model.predict(X_test).reshape(-1, 1)

# acc = Accuracy(pre_skl, y_test)
# print("SKLearn 分类准确度为 {}".format(acc))

In [78]:
# from Methods import NeuralNet as M_NN
# # 神经网络模型

# model_NN = M_NN.NeuralNetwork(X_train, y_train.flatten())
# model_NN.fit()
# pre = model_liner.predict(X_test)

# acc_1 = Accuracy(pre, Y_test)
# print("线性回归模型分类准确度为 {}".format(acc_1))

In [79]:
X_train, y_train, X_test, y_test = M_TOOL.random_Split_Data_Label(df)

def deep_model(feature_dim,label_dim):
    from keras.models import Sequential
    from keras.layers import Dense
    model = Sequential()
    print("create model. feature_dim ={}, label_dim ={}".format(feature_dim, label_dim))
    model.add(Dense(20, activation='relu', input_dim=feature_dim))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(label_dim, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


def train_deep(X_train,y_train,X_test,y_test):
    feature_dim = 12
    label_dim = 1
    model = deep_model(feature_dim,label_dim)
    model.summary()
    model.fit(X_train,y_train,batch_size=16, epochs=15,validation_data=(X_test,y_test))
    
train_deep(X_train,y_train,X_test,y_test)

create model. feature_dim =12, label_dim =1
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 20)                260       
                                                                 
 dense_7 (Dense)             (None, 20)                420       
                                                                 
 dense_8 (Dense)             (None, 20)                420       
                                                                 
 dense_9 (Dense)             (None, 20)                420       
                                                                 
 dense_10 (Dense)            (None, 20)                420       
                                                                 
 dense_11 (Dense)            (None, 1)                 21        
                                                                 
Total para

In [88]:
from Methods import XGBoost as M_XGB
# XGBoost 模型
X_train, y_train, X_test, y_test = M_TOOL.random_Split_Data_Label(df)

parameters = {
    "learning_rate": 0.3,       # 如同学习率
    "min_child_weight": 1, 
    # 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
    #，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
    # 这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。
    "max_depth": 9,             # 构建树的深度，越大越容易过拟合
    "gamma": 0,                 # 树的叶子节点上作进一步分区所需的最小损失减少,越大越保守，一般0.1、0.2这样子。
    "subsample": 1,             # 随机采样训练样本 训练实例的子采样比
    "max_delta_step": 0,        # 最大增量步长，我们允许每个树的权重估计。
    "colsample_bytree": 1,      # 生成树时进行的列采样 
    "reg_lambda": 1,            # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
    "reg_alpha": 0,             # L1 正则项参数
    "objective": 'multi:softmax',   # 多分类的问题 指定学习任务和相应的学习目标 binary:logitraw   multi:softmax
    "num_class": 4,             # 类别数
    "n_estimators": 90,         # 树的个数
    "seed": 2000,                # 随机种子     
    "if_silent": 1,
}


model_XGB = M_XGB.XGBoost(X_train, y_train)
model_XGB.fit(parameters)
# model_XGB.save("./model/Model_XGBoost_0.28")
# model_XGB= pk.load(open("./model/Model_XGBoost_0.28", "rb")) 

pre = model_XGB.predict(X_test)
pre = np.array(pre).reshape(-1, 1)

acc_xgb = Accuracy(pre, y_test)
print("XGBoost 模型分类准确度为 {}".format(acc_xgb))


XGBoost 模型分类准确度为 0.2805534204458109
